In [4]:
import pandas as pd
fp = "F:datasets/geo-pub-log-new-clean.csv"
df = pd.read_csv(fp)
#print(df.content.unique())
df.head()

,email,age,experience,pubs_as_1st_auth,pubs_as_2nd_auth,pubs_as_Others,Total_pubs,hits,duration,weekends,...,Most_Visited_journals_and_hits,Name,Grade,Mother_Tongue,Qualification,group,subgroup,division,section,PhD
0,aa_mani,55,23,NaN,NaN,8.0,8,5,150.6203,0,...,link.springer.com (4),MANI A.,SO/D,Tamil,DECE-ELECTR. AND COMMN,RFG,OMG,RMD,IMS,0
1,aacharya,33,8,1.0,NaN,0.0,1,2256,437.0053,0,...,economictimes.indiatimes.com (1372),AVINASH KUMAR ACHARYA,SO/D,Hindi,B.Tech-INST & CONTROL ENGG,FRTG,SFG,SED,EAS,0
2,aak,52,25,NaN,NaN,1.0,1,49,20.4817,0,...,www.astm.org (44),ANANTHA KUMAR A.,SO/E,Tamil,DME-MECHANICAL,MC&MFCG,MFCG,CFD,RFS,0
3,aakgen1,42,16,NaN,NaN,NaN,0,19532,3479.8417,0,...,economictimes.indiatimes.com (14502),ARUL ANANTHA KUMAR A,SO/F,Tamil,"M.SC-GENETICS,B.SC-AGRICULTURE",HSEG,RESG,RESD,RBDS,0
4,aamal,57,17,NaN,NaN,NaN,0,34,13.1981,0,...,www.astm.org (34),AMALRAJ A.,FM/B,Tamil,"Diploma in Mechanical Engineering-,H S C (10+2...",MC&MFCG,MFCG,FChD,IFSS,0


In [5]:
import nltk; nltk.download('stopwords')
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\ADMIN\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [6]:
fp = "F:datasets/geo-pub-log-new-clean.csv"
df = pd.read_csv(fp)
#print(df.content.unique())
df['experience2']=df['experience']**2

df.head()


,email,age,experience,pubs_as_1st_auth,pubs_as_2nd_auth,pubs_as_Others,Total_pubs,hits,duration,weekends,...,Name,Grade,Mother_Tongue,Qualification,group,subgroup,division,section,PhD,experience2
0,aa_mani,55,23,NaN,NaN,8.0,8,5,150.6203,0,...,MANI A.,SO/D,Tamil,DECE-ELECTR. AND COMMN,RFG,OMG,RMD,IMS,0,529
1,aacharya,33,8,1.0,NaN,0.0,1,2256,437.0053,0,...,AVINASH KUMAR ACHARYA,SO/D,Hindi,B.Tech-INST & CONTROL ENGG,FRTG,SFG,SED,EAS,0,64
2,aak,52,25,NaN,NaN,1.0,1,49,20.4817,0,...,ANANTHA KUMAR A.,SO/E,Tamil,DME-MECHANICAL,MC&MFCG,MFCG,CFD,RFS,0,625
3,aakgen1,42,16,NaN,NaN,NaN,0,19532,3479.8417,0,...,ARUL ANANTHA KUMAR A,SO/F,Tamil,"M.SC-GENETICS,B.SC-AGRICULTURE",HSEG,RESG,RESD,RBDS,0,256
4,aamal,57,17,NaN,NaN,NaN,0,34,13.1981,0,...,AMALRAJ A.,FM/B,Tamil,"Diploma in Mechanical Engineering-,H S C (10+2...",MC&MFCG,MFCG,FChD,IFSS,0,289


In [10]:
import statsmodels.api as sm
from patsy import dmatrices
formula1 = """Total_pubs ~ experience  + experience2"""
response, predictors = dmatrices(formula1, df, return_type='dataframe')
results = sm.GLM(response, predictors, family=sm.families.Poisson()).fit()
print(results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:             Total_pubs   No. Observations:                 1680
Model:                            GLM   Df Residuals:                     1677
Model Family:                 Poisson   Df Model:                            2
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2159.2
Date:                Wed, 10 Apr 2019   Deviance:                       3597.3
Time:                        14:18:08   Pearson chi2:                 8.32e+03
No. Iterations:                     6   Covariance Type:             nonrobust
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -2.1344      0.148    -14.429      0.000      -2.424      -1.844
experience      0.1241      0.016      7.771     

In [11]:
import statsmodels.api as sm
from patsy import dmatrices
formula1 = """Total_pubs ~ experience  + experience2 + C(PhD)"""
response, predictors = dmatrices(formula1, df, return_type='dataframe')
results = sm.GLM(response, predictors, family=sm.families.Poisson()).fit()
print(results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:             Total_pubs   No. Observations:                 1680
Model:                            GLM   Df Residuals:                     1676
Model Family:                 Poisson   Df Model:                            3
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1712.0
Date:                Wed, 10 Apr 2019   Deviance:                       2702.9
Time:                        14:18:21   Pearson chi2:                 7.10e+03
No. Iterations:                     6   Covariance Type:             nonrobust
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -2.1172      0.148    -14.264      0.000      -2.408      -1.826
C(PhD)[T.1]     2.2269      0.071     31.226     

In [12]:
import statsmodels.api as sm
from patsy import dmatrices
formula1 = """Total_pubs ~ experience  + experience2 + C(Grade)+C(PhD)"""
response, predictors = dmatrices(formula1, df, return_type='dataframe')
results = sm.GLM(response, predictors, family=sm.families.Poisson()).fit()
print(results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:             Total_pubs   No. Observations:                 1558
Model:                            GLM   Df Residuals:                     1502
Model Family:                 Poisson   Df Model:                           55
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1450.2
Date:                Wed, 10 Apr 2019   Deviance:                       2181.2
Time:                        14:18:33   Pearson chi2:                 4.43e+03
No. Iterations:                    24   Covariance Type:             nonrobust
                                         coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                            -24.8441   1.06e+05     -0.000      1.000   -2

In [13]:
import statsmodels.api as sm
from patsy import dmatrices
formula1 = """Total_pubs ~ experience  + experience2 + C(group)+C(PhD)"""
response, predictors = dmatrices(formula1, df, return_type='dataframe')
results = sm.GLM(response, predictors, family=sm.families.Poisson()).fit()
print(results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:             Total_pubs   No. Observations:                 1558
Model:                            GLM   Df Residuals:                     1538
Model Family:                 Poisson   Df Model:                           19
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1470.3
Date:                Wed, 10 Apr 2019   Deviance:                       2221.6
Time:                        14:31:04   Pearson chi2:                 5.14e+03
No. Iterations:                    23   Covariance Type:             nonrobust
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept             -24.8320   2.65e+04     -0.001      0.999    -5.2e+04    5.19e+04
C(group)[T.ADMIN]       0